# 04. 텍스트 분류 실습 - 20 뉴스그룹 분류

<br>

사이킷런이 내부에 가지고 있는 예제 데이터인 "20 뉴스그룹 데이터 세트"를 이용 텍스트 분류 적용.

텍스트 분류

- 특정 문서의 분류를 학습 데이터를 통해 학습해 모델 생성
- 이 학습 모델을 이용해 다른 문서의 분류를 예측

사이킷런은 `fetch_20newsgroups()` API를 이용해 뉴스그룹의 분류를 수행해 볼 수 있는 예제 데이터를 제공.

텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 된다.

이러한 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘

- 로지스틱 회귀
- 선형 서포트 벡터 머신
- 나이즈 베이즈

텍스트를 기반으로 분류를 수행할 때는 먼저 텍스트를 정규화한 뒤 피처 벡터화를 적용.

그리고 그 이후에 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가.

이번 절에서는 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용해 예측 성능을 비교.

피처 벡터화를 위한 파라미터와 `GridSearchCV` 기반의 하이퍼 파라미터 튜닝, 사이킷런의 `Pipeline` 객체를 통해 피처 벡터화 파라미터와 `GridSearchCV` 기반의 하이퍼 파라미터 튜닝을 한꺼번에 하는 방법 소개

<br>

## 4.1 텍스트 정규화

In [2]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

<br>

`fetch_20newsgroups()`는 파이썬 딕셔너리와 유사한 `Bunch` 객체를 반환.

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


`filenames` : `fetch_20newsgroups()` API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭

<br>

`Target` 클래서 구성 확인

In [4]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


<br>

`Target` 클래스의 값은 0부터 19까지 20개로 구성.

개별 데이터가 텍스트로 어떻게 구성돼 있는 지 데이터 한 개만 추출하여 확인

In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음.

이 중에서 내용을 제외하고 제목 등의 다른 정보는 제거.  
(제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 `Target` 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문)

`remove` 파라미터를 이용하면 뉴스그룹 기사의 헤더(header), 푸터(footer) 등을 제거할 수 있다.

또한 `fetch_20newsgroups()`는 `subset` 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# subset='train' 으로 학습용 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test' 으로 테스트 데이터만 추출
# remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len))